In [1]:
from spectral_cube_plus import SpectralCubePlus
from astropy import units as u
from glob import glob 

# Individual Masking

In [2]:
input_files = glob(f"{DATA_PATH}/Galactic/NOEMA/data/CloudH_lines/*.fits")
input_files

['/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_HNC_L27-merged.fits',
 '/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_HCN_L22-merged.fits',
 '/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_N2Dp_L18-merged.fits',
 '/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_DCOp_L09-merged.fits',
 '/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_DNC_L16-merged.fits',
 '/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_H15NC_L23-merged.fits',
 '/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_HCOp_L25-merged.fits',
 '/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_DCN_L10-merged.fits',
 '/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_line

In [7]:
restfreqs = {'N2Hp': 93.1733977 * u.GHz,
             'N2Dp': 77.1092433 * u.GHz,}

for input_file in input_files:
    
    print(input_file)

    cube = SpectralCubePlus.read(input_file)
    cube.allow_huge_operations = True
    if 'N2Hp' in input_file:
        print('   N2Hp - changing rest frequency')
        cube = cube.with_spectral_unit(u.km / u.s, rest_value=restfreqs['N2Hp']) 
    elif 'N2Dp' in input_file:
        print('   N2Dp - changing rest frequency')
        cube = cube.with_spectral_unit(u.km / u.s, rest_value=restfreqs['N2Dp'])
    else: 
        continue
        print('No rest frequency change')
        cube = cube.with_spectral_unit(u.km / u.s) 
    cube = cube.to('K')
    cube = cube.minimal_subcube()

    cube.writeto(input_file, overwrite=True)

/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_HNC_L27-merged.fits
/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_HCN_L22-merged.fits
/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_N2Dp_L18-merged.fits
   N2Dp - changing rest frequency
/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_DCOp_L09-merged.fits
/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_DNC_L16-merged.fits
/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_H15NC_L23-merged.fits
/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_HCOp_L25-merged.fits
/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_DCN_L10-merged.fits
/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_line

In [4]:
# for input_file in input_files:
    
#     print(input_file)
#     output_file = input_file.replace('CloudH_lines', 'CloudH_lines_moms') 

#     cube = SpectralCubePlus.read(input_file)
#     cube.allow_huge_operations = True
#     cube = cube.with_spectral_unit(u.km / u.s) 
#     cube = cube.to('K')

#     rms = cube.get_rms_auto()
#     mask = cube < cube.rms
#     rms = cube.get_rms_auto(mask)

#     _ = cube.get_expmask(hthresh=4, lthresh=2)

#     mom8 = cube.masked.max(axis=0)
#     mom0 = cube.masked.moment0()
#     mom0err = cube.masked.moment0err()

#     mom8.write(output_file.replace('.fits', '_max.fits'), overwrite=True)
#     mom0.write(output_file.replace('.fits', '_mom0.fits'), overwrite=True)
#     mom0err.write(output_file.replace('.fits', '_mom0err.fits'), overwrite=True)

# N2H+ Isolated

In [9]:
file_n2hp = '/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_N2Hp_L30-merged.fits'

In [ ]:
def get_moms(input_file, velo_slab=None):

    print(input_file)
    output_file = input_file.replace('CloudH_lines', 'CloudH_lines_moms') 

    cube = SpectralCubePlus.read(input_file)
    cube.allow_huge_operations = True
    cube = cube.with_spectral_unit(u.km / u.s) 
    cube = cube.to('K')

    rms = cube.get_rms_auto()
    mask = cube < cube.rms
    rms = cube.get_rms_auto(mask)
    rms_hdu = cube.rms_hdu

    if velo_slab is not None:
        # Limit to 40 to 50 km/s - see in Hernandez et al. 2011
        cube = cube.spectral_slab(velo_slab[0]*u.km/u.s, velo_slab[1]*u.km/u.s)
        cube.rms = rms
        cube.rms_hdu = rms_hdu

    _ = cube.get_expmask(hthresh=4, lthresh=2)

    mom8 = cube.masked.max(axis=0)
    mom0 = cube.masked.moment0()
    mom0err = cube.masked.moment0err()

    mom8.write(output_file.replace('.fits', '_iso_max.fits'), overwrite=True)
    mom0.write(output_file.replace('.fits', '_iso_mom0.fits'), overwrite=True)
    mom0err.write(output_file.replace('.fits', '_iso_mom0err.fits'), overwrite=True)

    return cube 

cube_n2hp = get_moms(file_n2hp, velo_slab=[30, 40])

/Users/abarnes/Library/CloudStorage/Dropbox/Data//Galactic/NOEMA/data/CloudH_lines/CloudH_N2Hp_L30-merged.fits


/opt/anaconda3/lib/python3.12/site-packages/spectral_cube/spectral_cube.py:436: RuntimeWarning: All-NaN slice encountered
  out = function(self._get_filled_data(fill=fill,
